In [0]:
# ✅ Step 1: Define your Azure Storage info
storage_account_name = "dmodemoaccount123"
storage_account_key = ""  # Replace with actual key from Azure Portal

# ✅ Step 2: Set Spark config for accessing ADLS Gen2
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)

In [0]:
sourceCSVFileURL="https://retailpricing.blob.core.windows.net/daily-pricing"
bronzelayerCSVFilePath="abfss://working-labs@dmodemoaccount123.dfs.core.windows.net/bronze/daily-pricing/csv"

In [0]:
dailyPricingSourceBaseURL="https://retailpricing.blob.core.windows.net/"
dailyPricingSourceFolder="daily-pricing/"
dailyPricingSourceFileName="PW_MW_DR_01012023.csv"

dailyPricingSinkLayerName="bronze"
dailyPricingSinkStorageAccountName=storage_account_name
dailyPricingSinkFolderName="daily-pricing"

In [0]:
import pandas as pds

In [0]:
dailyPricingSourceURL=dailyPricingSourceBaseURL+dailyPricingSourceFolder+dailyPricingSourceFileName
print(dailyPricingSourceURL)

https://retailpricing.blob.core.windows.net/daily-pricing/PW_MW_DR_01012023.csv


In [0]:
pds.read_csv(dailyPricingSourceURL)


,DATE_OF_PRICING,ROW_ID,STATE_NAME,MARKET_NAME,PRODUCTGROUP_NAME,PRODUCT_NAME,VARIETY,ORIGIN,ARRIVAL_IN_TONNES,MINIMUM_PRICE,MAXIMUM_PRICE,MODAL_PRICE
0,01/01/2023,12699,Uttar Pradesh,Najibabad,Fruits,Raddish,Raddish,NR,23,250,350,300.0
1,01/01/2023,12711,Uttar Pradesh,Sahiyapur,Fruits,Raddish,Raddish,NR,2.6,330,415,370.0
2,01/01/2023,12723,Uttar Pradesh,Ujhani,Fruits,Raddish,Other,NR,0.6,450,550,500.0
3,01/01/2023,12735,Uttrakhand,Rudrapur,Fruits,Raddish,Other,NR,2.5,100,300,200.0
4,01/01/2023,12759,Kerala,Manjeswaram,Fruits,Ridge gourd(Tori),Ridgeguard(Tori),NR,5,3400,4400,3900.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12872,01/01/2023,12753,Karnataka,Hospet,Fruits,Ridge gourd(Tori),Ridgeguard(Tori),NR,2,1200,1400,1300.0
12873,01/01/2023,12765,Maharashtra,Mumbai,Fruits,Ridge gourd(Tori),Other,NR,19,2000,2600,2300.0
12874,01/01/2023,12777,West Bengal,Balarampur,Fruits,Ridge gourd(Tori),Other,NR,0.04,3000,3100,3050.0
12875,01/01/2023,12789,Himachal Pradesh,Mandi(Takoli),Fruits,Season Leaves,Other,NR,0.1,500,700,600.0


In [0]:
dailyPricingPandasDF=pds.read_csv(dailyPricingSourceURL)
dailyPricingPandasDF.head()

,DATE_OF_PRICING,ROW_ID,STATE_NAME,MARKET_NAME,PRODUCTGROUP_NAME,PRODUCT_NAME,VARIETY,ORIGIN,ARRIVAL_IN_TONNES,MINIMUM_PRICE,MAXIMUM_PRICE,MODAL_PRICE
0,01/01/2023,12699,Uttar Pradesh,Najibabad,Fruits,Raddish,Raddish,NR,23,250,350,300.0
1,01/01/2023,12711,Uttar Pradesh,Sahiyapur,Fruits,Raddish,Raddish,NR,2.6,330,415,370.0
2,01/01/2023,12723,Uttar Pradesh,Ujhani,Fruits,Raddish,Other,NR,0.6,450,550,500.0
3,01/01/2023,12735,Uttrakhand,Rudrapur,Fruits,Raddish,Other,NR,2.5,100,300,200.0
4,01/01/2023,12759,Kerala,Manjeswaram,Fruits,Ridge gourd(Tori),Ridgeguard(Tori),NR,5,3400,4400,3900.0


In [0]:
dailyPricingSparkDF=spark.createDataFrame(dailyPricingPandasDF)
dailyPricingSparkDF.printSchema()

root
 |-- DATE_OF_PRICING: string (nullable = true)
 |-- ROW_ID: long (nullable = true)
 |-- STATE_NAME: string (nullable = true)
 |-- MARKET_NAME: string (nullable = true)
 |-- PRODUCTGROUP_NAME: string (nullable = true)
 |-- PRODUCT_NAME: string (nullable = true)
 |-- VARIETY: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ARRIVAL_IN_TONNES: string (nullable = true)
 |-- MINIMUM_PRICE: string (nullable = true)
 |-- MAXIMUM_PRICE: string (nullable = true)
 |-- MODAL_PRICE: double (nullable = true)



In [0]:
dailyPricingSinkFolderPath=f"abfss://{dailyPricingSinkLayerName}@{dailyPricingSinkStorageAccountName}.dfs.core.windows.net/{dailyPricingSinkFolderName}"
print(dailyPricingSinkFolderPath)
#dailyPricingSparkDF.write.mode("overwrite").csv(dailyPricingSinkFolderPath)

(
    dailyPricingSparkDF
    .write
    .mode("overwrite")
    .option("header","true")
    .csv(dailyPricingSinkFolderPath)
)

abfss://bronze@dmodemoaccount123.dfs.core.windows.net/daily-pricing


In [0]:
processName='dailyPricingSourceIngest'

In [0]:
dbutils.widgets.text('prm_dailyPricingSourceFileDate','')
print(dbutils.widgets.get('prm_dailyPricingSourceFileDate'))

2025-03-01


In [0]:
%sql
--
--DROP TABLE IF EXISTS processrunlogs.processrunlog
CREATE SCHEMA IF NOT Exists processrunlogs



In [0]:
%sql
CREATE TABLE IF NOT EXISTS processrunlogs.deltalakehouse_process_run(
PROCESS_NAME STRING,
PROCESS_FILE_TABLE_DATE STRING,
PROCESS_STATUS STRING
)


In [0]:
processName='dailyPricingSourceIngest'
processFileDate=dbutils.widgets.get('prm_dailyPricingSourceFileDate')
processStatus='Completed'

processInsertSql = f"""
INSERT INTO processrunlogs.deltalakehouse_process_run 
(PROCESS_NAME, PROCESS_FILE_TABLE_DATE, PROCESS_STATUS)
VALUES ('{processName}', '{processFileDate}', '{processStatus}')
"""

spark.sql(processInsertSql)


DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
DROP TABLE processrunlogs.deltalakehouse_process_run

In [0]:
%sql
select * from processrunlogs.deltalakehouse_process_run


PROCESS_NAME,PROCESS_FILE_TABLE_DATE,PROCESS_STATUS
dailyPricingSourceIngest,2025-03-01,Completed
dailyPricingSourceIngest,2025-01-01,Completed
dailyPricingSourceIngest,2025-02-01,Completed


In [0]:
%sql
SELECT DATE_ADD(MAX(CAST(PROCESS_FILE_TABLE_DATE AS DATE)), 1) as NEXT_SOURCE_FILE_DATE
FROM processrunlogs.deltalakehouse_process_run where PROCESS_STATUS='Completed' and PROCESS_NAME='dailyPricingSourceIngest'

NEXT_SOURCE_FILE_DATE
2025-03-02
